In [13]:
import openai
import yaml
import wandb
import sys, os
import tiktoken

from IPython.display import Image

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from src.utils import *
from src.svg_quality_checks import *
from src.gpt_wrappers import *

# load yaml config
with open("../config.yml", "r") as f:
    config = yaml.safe_load(f)

openai.api_key = config["OPENAI_KEY"]

# Few-shot learning (give examples)

In [14]:
system_prompt = """
    You are an expert in design in different fields, you use different vector graphics tools.
    You deal professionally with SVG files, you know how to create complex design with them.
    Your main job is to complete design systems and tokens based on the initial inputs. 
    These initial inputs are SVG files that can consist of multiple layers, groups, paths, etc.
    They can be different elements as icons, logos, buttons, checkboxes, etc.
"""

## Case 1: 
### Case 1.1: 

In [15]:
FILENAME_INPUT = "segmented_buttons_short"
svg_shot_input = load_raw_svg_optimize_clean_save_png(FILENAME_INPUT)

FILENAME_OUTPUT = "segmented_buttons_short_middle_on"
svg_shot_output = load_raw_svg_optimize_clean_save_png(FILENAME_OUTPUT)


segmented_buttons_short_optimized.svg:
Done in 59 ms!
13.162 KiB - 70.3% = 3.909 KiB

segmented_buttons_short_middle_on_optimized.svg:
Done in 64 ms!
13.429 KiB - 69.4% = 4.105 KiB


In [16]:
encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")
len(encoder.encode(svg_shot_input)), len(encoder.encode(svg_shot_output))

(1425, 1515)

In [17]:
init_question_prompt = f"""
    This is XML for the SVG file with app bottom bar buttons: \n

    {svg_shot_input}

    Generate an SVG where the middle button is on, has check sign on the left and says Selected.

    Keep it as short and optimized as you can, because you have limit of tokens.
    Start with <svg ... and end with </svg>

"""
init_answer_prompt = f"{svg_shot_output}"

In [18]:
question = f"""
    Generate an SVG where the left button is on, has check sign on the left and says Selected.
"""

In [19]:
chatgpt = VanillaChatGPTWrapper(system_prompt)
chatgpt.initialize_with_question_answer(init_question_prompt, init_answer_prompt)
answer = chatgpt.generate(question)
print("Quality check:", validate_svg(answer), get_svg_size(answer))

ParseError: unclosed token: line 1, column 2338 (<string>)

In [20]:
answer

'<svg width="258" height="48" fill="none" style="-webkit-print-color-adjust:exact"><defs><clipPath id="a" class="frame-clip-def frame-clip"><rect width="258" height="48" rx="0" ry="0"/></clipPath></defs><g clip-path="url(#a)"><g class="fills"><rect width="258" height="48" class="frame-background" rx="0" ry="0"/></g><g class="frame-children"><defs><clipPath id="b" class="frame-clip-def frame-clip"><rect width="250" height="40" x="4" y="4" rx="0" ry="0"/></clipPath></defs><g clip-path="url(#b)"><g class="fills"><rect width="250" height="40" x="4" y="4" class="frame-background" rx="0" ry="0"/></g><g class="frame-children"><g class="fills"><rect width="73" height="40" x="4" y="4" class="frame-background" rx="0" ry="0"/></g><g class="frame-children"><defs><clipPath id="c" class="frame-clip-def frame-clip"><path d="M3 4h54a20 20 0 0 1 20 20 20 20 0 0 1-20 20H3V4z"/></clipPath></defs><g clip-path="url(#c)"><g class="fills"><path d="M3 4h54a20 20 0 0 1 20 20 20 20 0 0 1-20 20H3V4z" class="fram

In [21]:
len(encoder.encode(init_question_prompt)) + \
len(encoder.encode(init_answer_prompt)) + \
len(encoder.encode(question)) + \
len(encoder.encode(answer))

3982